In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = "3" # choose which GPU you want to use

from tempfile import gettempdir
import urllib
import zipfile
import numpy as np
import random
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

1.11.0


# Learning the XOR

In [2]:
# xor task
xor_data = np.array([[1, 0],
                    [0, 1],
                    [1, 1],
                    [0, 0]])
xor_label = np.array([[1], [1], [0], [0]])

In [15]:
learning_rate = 0.2
max_epochs = 10000
skip_step = 500

tf.reset_default_graph()

with tf.Graph().as_default() as g:
    
    x_input = tf.placeholder(tf.float32, [None, 2])
    y_label = tf.placeholder(tf.float32, [None, 1])
    
    # start building your model and meet the requirements
    # from here
    
    with tf.name_scope("model"):
        w1 = tf.get_variable(name="w1", shape=[2, 2],
                          initializer=tf.random_normal_initializer)
        b1 = tf.get_variable(name="b1", shape=[2],
                          initializer=tf.random_normal_initializer)
        
        logits = tf.matmul(x_input, w1) + b1
        logits = tf.exp(logits) / (1 + tf.exp(logits)) #sigmoid
        
        w2 = tf.get_variable(name="w2", shape=[2,1],
                          initializer=tf.random_normal_initializer)
        b2 = tf.get_variable(name="b2", shape=[1],
                          initializer=tf.random_normal_initializer)
        
        z = tf.matmul(logits, w2) + b2
        z = tf.exp(z) / (1 + tf.exp(z)) #sigmoid

        output = tf.round(z)

        init_op = tf.global_variables_initializer()
        
        
    with tf.name_scope("loss"):
        error = tf.square(y_label - z)
        loss = tf.reduce_sum(error)
    
    
    with tf.name_scope("optimizer"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    
with tf.Session(graph=g) as sess:
    # start run the seesion and meet the requrements
    # from here
    sess.run(init_op)
    
    total_loss = 0.0 

    print("[start training]")
    for index in range(max_epochs):
        y_hat, out_loss, _ = sess.run([output, loss, optimizer], feed_dict={x_input: xor_data, 
                                                           y_label: xor_label})
        total_loss += out_loss
        if (index+1) % skip_step == 0:
            print('Average loss at step {}: {:5.1f}'.format(
                index+1, total_loss / skip_step))
            total_loss = 0.0
    print("[end training]")
            
    print("\n\npredicted output of four cases : \n", y_hat)
    accuracy  = np.sum(xor_label == y_hat) / 4
    print("Accuracy : %.1f" %accuracy)
    
    
    print("\n\nValue of weights and bias")
    w1, w2, b1, b2 = sess.run([w1, w2, b1, b2])
    print("Weight 1 : \n", w1)
    print("Bias 1 : ", b1)
    print("Weight 2 : \n", w2)
    print("Bias 2 : ", b2)

[start training]
Average loss at step 500:   1.0
Average loss at step 1000:   0.7
Average loss at step 1500:   0.1
Average loss at step 2000:   0.0
Average loss at step 2500:   0.0
Average loss at step 3000:   0.0
Average loss at step 3500:   0.0
Average loss at step 4000:   0.0
Average loss at step 4500:   0.0
Average loss at step 5000:   0.0
Average loss at step 5500:   0.0
Average loss at step 6000:   0.0
Average loss at step 6500:   0.0
Average loss at step 7000:   0.0
Average loss at step 7500:   0.0
Average loss at step 8000:   0.0
Average loss at step 8500:   0.0
Average loss at step 9000:   0.0
Average loss at step 9500:   0.0
Average loss at step 10000:   0.0
[end training]


predicted output of four cases : 
 [[1.]
 [1.]
 [0.]
 [0.]]
Accuracy : 1.0


Value of weights and bias
Weight 1 : 
 [[-6.338139   5.5041456]
 [ 6.283338  -5.759913 ]]
Bias 1 :  [-3.52308   -3.0566976]
Weight 2 : 
 [[9.028484]
 [9.20646 ]]
Bias 2 :  [-4.5037456]


# MNIST - Digits Classification

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
# load mnist data
mnist = input_data.read_data_sets("data/mnist", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [21]:
learning_rate = 0.01
max_epochs = 500
batch_size = 32

tf.reset_default_graph()

with tf.Graph().as_default() as g:
    
    # start building your model and meet the requirements
    # from here
    with tf.name_scope("data"):
        x_input = tf.placeholder(tf.float32, [None, 28*28])
        y_label = tf.placeholder(tf.float32, [None, 10])
        
        training_dataset = tf.data.Dataset.from_tensor_slices((x_input, y_label))
        training_dataset = training_dataset.batch(batch_size)
        training_iterator = training_dataset.make_initializable_iterator()
        
        test_dataset = tf.data.Dataset.from_tensor_slices((x_input, y_label))
        test_iterator = training_dataset.make_initializable_iterator()

        handle = tf.placeholder(tf.string, shape=[])
        iter = tf.data.Iterator.from_string_handle(
            handle, training_dataset.output_types, training_dataset.output_shapes)
        x, y = iter.get_next()
        
    with tf.name_scope("model"):
        #fc1
        w1 = tf.get_variable("w1", shape=[28*28, 100], initializer=tf.random_normal_initializer)
        b1 = tf.get_variable("b1", shape=[100], initializer=tf.random_normal_initializer)
        middle = tf.matmul(x, w1) + b1

        #fc2
        w2 = tf.get_variable("w2", shape=[100, 10], initializer=tf.random_normal_initializer)
        b2 = tf.get_variable("b2", shape=[10], initializer=tf.random_normal_initializer)
        z = tf.matmul(middle, w2) + b2
        
        #softmax
        z = z - tf.reshape(tf.reduce_max(z, axis=1), (-1,1) )
        out = tf.exp(z) / tf.reshape(tf.reduce_sum(tf.exp(z), axis=1), (-1,1) )
        
        output = tf.argmax(out, axis=1)
        
        init_op = tf.global_variables_initializer()
        
        
    with tf.name_scope("loss"):
        error = tf.square(y - out)
        loss = tf.reduce_sum(error)
        
        
    with tf.name_scope("Accuracy"):
        correct_prediction = tf.equal(output, tf.argmax(y, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        
    with tf.name_scope("optimizer"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        

with tf.Session(graph=g) as sess:
    # start run the seesion and meet the requrements
    # from here 
    sess.run(init_op)
    
    train_handle = sess.run(training_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    print("[start training]")
    
    total_loss = 0.0 
    total_acc = 0.0
    old_loss = 1000.0
    for index in range(max_epochs):
        
        sess.run(training_iterator.initializer, feed_dict={ x_input: mnist.train.images, y_label: mnist.train.labels})
        sess.run(test_iterator.initializer, feed_dict={ x_input: mnist.validation.images, y_label: mnist.validation.labels})
    
        while True:
            try:
                acc, out_loss, _ = sess.run([accuracy, loss, optimizer], feed_dict = {handle: train_handle})
                total_loss += out_loss
                total_acc += acc*batch_size
                
            except tf.errors.OutOfRangeError:
                print('Epoch {:3} : loss = {:5.4f}, accuracy = {:2.3f}'.format(index+1, total_loss/mnist.train.num_examples, total_acc/mnist.train.num_examples), end=" ")
                total_loss = 0.0
                total_acc = 0.0
                break
        
        #validation
        acc, out_loss = sess.run([accuracy, loss], feed_dict = {handle: test_handle})
        print('| valid_accuracy = {:2.3f} | lr = {:.5f}'.format(acc, learning_rate))
        if out_loss >= old_loss*0.9:
            learning_rate *= 0.9
        old_loss = out_loss
    
    print("[end training]")
    
    #final test
    print("\n[testing]")
    sess.run(test_iterator.initializer, feed_dict={ x_input: mnist.test.images, y_label: mnist.test.labels})
    acc, out_loss = sess.run([accuracy, loss], feed_dict = {handle: test_handle})
    print('loss = {:1.3f} , accuracy = {:2.3f}'.format(out_loss, acc))
    

[start training]
Epoch   1 : loss = 0.9877, accuracy = 0.503 | valid_accuracy = 0.625 | lr = 0.01000
Epoch   2 : loss = 0.7352, accuracy = 0.630 | valid_accuracy = 0.625 | lr = 0.01000
Epoch   3 : loss = 0.7048, accuracy = 0.646 | valid_accuracy = 0.625 | lr = 0.00900
Epoch   4 : loss = 0.6810, accuracy = 0.658 | valid_accuracy = 0.656 | lr = 0.00810
Epoch   5 : loss = 0.5814, accuracy = 0.708 | valid_accuracy = 0.812 | lr = 0.00729
Epoch   6 : loss = 0.5324, accuracy = 0.732 | valid_accuracy = 0.844 | lr = 0.00729
Epoch   7 : loss = 0.4279, accuracy = 0.785 | valid_accuracy = 0.875 | lr = 0.00729
Epoch   8 : loss = 0.3763, accuracy = 0.811 | valid_accuracy = 0.875 | lr = 0.00729
Epoch   9 : loss = 0.2919, accuracy = 0.853 | valid_accuracy = 0.906 | lr = 0.00656
Epoch  10 : loss = 0.2611, accuracy = 0.868 | valid_accuracy = 0.906 | lr = 0.00656
Epoch  11 : loss = 0.2467, accuracy = 0.876 | valid_accuracy = 0.906 | lr = 0.00590
Epoch  12 : loss = 0.2392, accuracy = 0.880 | valid_accurac

Epoch  99 : loss = 0.1761, accuracy = 0.912 | valid_accuracy = 0.938 | lr = 0.00001
Epoch 100 : loss = 0.1768, accuracy = 0.911 | valid_accuracy = 0.938 | lr = 0.00001
Epoch 101 : loss = 0.1802, accuracy = 0.910 | valid_accuracy = 0.906 | lr = 0.00001
Epoch 102 : loss = 0.1818, accuracy = 0.909 | valid_accuracy = 0.906 | lr = 0.00001
Epoch 103 : loss = 0.1868, accuracy = 0.906 | valid_accuracy = 0.844 | lr = 0.00001
Epoch 104 : loss = 0.1796, accuracy = 0.910 | valid_accuracy = 0.906 | lr = 0.00001
Epoch 105 : loss = 0.1836, accuracy = 0.908 | valid_accuracy = 0.875 | lr = 0.00001
Epoch 106 : loss = 0.1934, accuracy = 0.903 | valid_accuracy = 0.906 | lr = 0.00001
Epoch 107 : loss = 0.1731, accuracy = 0.913 | valid_accuracy = 0.906 | lr = 0.00001
Epoch 108 : loss = 0.1833, accuracy = 0.908 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 109 : loss = 0.1821, accuracy = 0.909 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 110 : loss = 0.1883, accuracy = 0.906 | valid_accuracy = 0.906 | lr = 

Epoch 197 : loss = 0.1798, accuracy = 0.910 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 198 : loss = 0.1647, accuracy = 0.918 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 199 : loss = 0.1630, accuracy = 0.918 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 200 : loss = 0.1793, accuracy = 0.910 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 201 : loss = 0.1760, accuracy = 0.912 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 202 : loss = 0.1706, accuracy = 0.915 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 203 : loss = 0.1741, accuracy = 0.913 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 204 : loss = 0.1674, accuracy = 0.916 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 205 : loss = 0.1665, accuracy = 0.917 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 206 : loss = 0.1667, accuracy = 0.917 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 207 : loss = 0.1679, accuracy = 0.916 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 208 : loss = 0.1747, accuracy = 0.913 | valid_accuracy = 0.938 | lr = 

Epoch 295 : loss = 0.1686, accuracy = 0.916 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 296 : loss = 0.1569, accuracy = 0.922 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 297 : loss = 0.1685, accuracy = 0.916 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 298 : loss = 0.1816, accuracy = 0.909 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 299 : loss = 0.1682, accuracy = 0.916 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 300 : loss = 0.1776, accuracy = 0.911 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 301 : loss = 0.1751, accuracy = 0.912 | valid_accuracy = 0.875 | lr = 0.00000
Epoch 302 : loss = 0.1727, accuracy = 0.914 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 303 : loss = 0.1723, accuracy = 0.914 | valid_accuracy = 0.875 | lr = 0.00000
Epoch 304 : loss = 0.1784, accuracy = 0.911 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 305 : loss = 0.1699, accuracy = 0.915 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 306 : loss = 0.1725, accuracy = 0.914 | valid_accuracy = 0.969 | lr = 

Epoch 393 : loss = 0.1652, accuracy = 0.917 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 394 : loss = 0.1609, accuracy = 0.920 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 395 : loss = 0.1718, accuracy = 0.914 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 396 : loss = 0.1674, accuracy = 0.916 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 397 : loss = 0.1637, accuracy = 0.918 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 398 : loss = 0.1721, accuracy = 0.914 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 399 : loss = 0.1685, accuracy = 0.916 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 400 : loss = 0.1687, accuracy = 0.916 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 401 : loss = 0.1669, accuracy = 0.917 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 402 : loss = 0.1770, accuracy = 0.912 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 403 : loss = 0.1636, accuracy = 0.918 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 404 : loss = 0.1586, accuracy = 0.921 | valid_accuracy = 0.969 | lr = 

Epoch 491 : loss = 0.1776, accuracy = 0.911 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 492 : loss = 0.1613, accuracy = 0.919 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 493 : loss = 0.1715, accuracy = 0.914 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 494 : loss = 0.1758, accuracy = 0.912 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 495 : loss = 0.1781, accuracy = 0.911 | valid_accuracy = 0.938 | lr = 0.00000
Epoch 496 : loss = 0.1786, accuracy = 0.911 | valid_accuracy = 0.969 | lr = 0.00000
Epoch 497 : loss = 0.1703, accuracy = 0.915 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 498 : loss = 0.1639, accuracy = 0.918 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 499 : loss = 0.1805, accuracy = 0.910 | valid_accuracy = 0.906 | lr = 0.00000
Epoch 500 : loss = 0.1645, accuracy = 0.918 | valid_accuracy = 0.938 | lr = 0.00000
[end training]

[testing]
loss = 2.000 , accuracy = 0.969


In [85]:
learning_rate = 0.1
max_epochs = 1000
batch_size = 128

tf.reset_default_graph()

with tf.Graph().as_default() as g:
    
    # start building your model and meet the requirements
    # from here
    with tf.name_scope("data"):
        x_input = tf.placeholder(tf.float32, [None, 28*28])
        y_label = tf.placeholder(tf.float32, [None, 10])
        lr = tf.placeholder(tf.float32, [])
        
        training_dataset = tf.data.Dataset.from_tensor_slices((x_input, y_label))
        training_dataset = training_dataset.batch(batch_size)
        training_iterator = training_dataset.make_initializable_iterator()
        
        test_dataset = tf.data.Dataset.from_tensor_slices((x_input, y_label))
        test_iterator = training_dataset.make_initializable_iterator()

        handle = tf.placeholder(tf.string, shape=[])
        iter = tf.data.Iterator.from_string_handle(
            handle, training_dataset.output_types, training_dataset.output_shapes)
        x, y = iter.get_next()
        
    with tf.name_scope("model"):
        #fc1
        w1 = tf.get_variable("w1", shape=[28*28, 10], initializer=tf.random_normal_initializer)
        b1 = tf.get_variable("b1", shape=[10], initializer=tf.random_normal_initializer)
        z = tf.matmul(x, w1) + b1
        
        #softmax
        z = z - tf.reshape(tf.reduce_max(z, axis=1), (-1,1) )
        out = tf.exp(z) / tf.reshape(tf.reduce_sum(tf.exp(z), axis=1), (-1,1) )
        
        output = tf.argmax(out, axis=1)
        
        init_op = tf.global_variables_initializer()
        
        
    with tf.name_scope("loss"):
        error = tf.abs(y - out)
        loss = tf.reduce_mean( tf.reduce_sum(error, axis=1) )
        #loss = -tf.reduce_sum( y * tf.log(out) )
        
    with tf.name_scope("Accuracy"):
        correct_prediction = tf.equal(output, tf.argmax(y, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        
    with tf.name_scope("optimizer"):
        optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
        

with tf.Session(graph=g) as sess:
    # start run the seesion and meet the requrements
    # from here 
    sess.run(init_op)
    
    train_handle = sess.run(training_iterator.string_handle())
    test_handle = sess.run(test_iterator.string_handle())
    print("[start training]")
    
    total_loss = 0.0 
    total_acc = 0.0
    old_loss = 1000.0
    for index in range(max_epochs):
        
        sess.run(training_iterator.initializer, feed_dict={ x_input: mnist.train.images, y_label: mnist.train.labels})
        sess.run(test_iterator.initializer, feed_dict={ x_input: mnist.validation.images, y_label: mnist.validation.labels})
    
        while True:
            try:
                acc, out_loss, _ = sess.run([accuracy, loss, optimizer], feed_dict = {handle: train_handle, lr: learning_rate})
                total_loss += out_loss*batch_size
                total_acc += acc*batch_size
                
            except tf.errors.OutOfRangeError:
                print('Epoch {:3} : loss = {:5.4f}, accuracy = {:2.3f}'.format(index+1, total_loss/mnist.train.num_examples, total_acc/mnist.train.num_examples), end=" ")
                total_loss = 0.0
                total_acc = 0.0
                break
        
        #validation
        acc, out_loss = sess.run([accuracy, loss], feed_dict = {handle: test_handle})
        print('| valid_accuracy = {:2.3f} | lr = {:.4f}'.format(acc, learning_rate))
        #if index > 20 and out_loss > old_loss and learning_rate > 1e-3:
        #    learning_rate *= 0.9
        #old_loss = out_loss
        
        if acc > 0.99 and index >= 100:
            print("[Early Stop]")
            break
    
    print("[end training]")
    
    #final test
    print("\n[testing]")
    sess.run(test_iterator.initializer, feed_dict={ x_input: mnist.test.images, y_label: mnist.test.labels})
    acc, out_loss = sess.run([accuracy, loss], feed_dict = {handle: test_handle})
    print('loss = {:1.4f} , accuracy = {:2.3f}'.format(out_loss/mnist.test.num_examples, acc))

[start training]
Epoch   1 : loss = 1.6054, accuracy = 0.201 | valid_accuracy = 0.328 | lr = 0.1000
Epoch   2 : loss = 1.4188, accuracy = 0.295 | valid_accuracy = 0.383 | lr = 0.1000
Epoch   3 : loss = 1.3486, accuracy = 0.330 | valid_accuracy = 0.391 | lr = 0.1000
Epoch   4 : loss = 1.2921, accuracy = 0.358 | valid_accuracy = 0.445 | lr = 0.1000
Epoch   5 : loss = 1.2313, accuracy = 0.389 | valid_accuracy = 0.461 | lr = 0.1000
Epoch   6 : loss = 1.1700, accuracy = 0.420 | valid_accuracy = 0.477 | lr = 0.1000
Epoch   7 : loss = 1.1198, accuracy = 0.445 | valid_accuracy = 0.477 | lr = 0.1000
Epoch   8 : loss = 1.0827, accuracy = 0.464 | valid_accuracy = 0.484 | lr = 0.1000
Epoch   9 : loss = 1.0553, accuracy = 0.477 | valid_accuracy = 0.500 | lr = 0.1000
Epoch  10 : loss = 1.0332, accuracy = 0.487 | valid_accuracy = 0.492 | lr = 0.1000
Epoch  11 : loss = 1.0148, accuracy = 0.497 | valid_accuracy = 0.523 | lr = 0.1000
Epoch  12 : loss = 0.9995, accuracy = 0.505 | valid_accuracy = 0.539 |

Epoch 100 : loss = 0.7067, accuracy = 0.651 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 101 : loss = 0.7063, accuracy = 0.651 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 102 : loss = 0.7059, accuracy = 0.651 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 103 : loss = 0.7055, accuracy = 0.651 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 104 : loss = 0.7051, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 105 : loss = 0.7048, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 106 : loss = 0.7044, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 107 : loss = 0.7040, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 108 : loss = 0.7037, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 109 : loss = 0.7033, accuracy = 0.652 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 110 : loss = 0.7030, accuracy = 0.653 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 111 : loss = 0.7027, accuracy = 0.653 | valid_accuracy = 0.641 | lr = 0.1000
Epoc

Epoch 199 : loss = 0.6848, accuracy = 0.662 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 200 : loss = 0.6847, accuracy = 0.662 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 201 : loss = 0.6846, accuracy = 0.662 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 202 : loss = 0.6844, accuracy = 0.662 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 203 : loss = 0.6843, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 204 : loss = 0.6841, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 205 : loss = 0.6840, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 206 : loss = 0.6839, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 207 : loss = 0.6837, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 208 : loss = 0.6836, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 209 : loss = 0.6834, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoch 210 : loss = 0.6833, accuracy = 0.663 | valid_accuracy = 0.641 | lr = 0.1000
Epoc

Epoch 298 : loss = 0.5407, accuracy = 0.735 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 299 : loss = 0.5404, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 300 : loss = 0.5401, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 301 : loss = 0.5399, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 302 : loss = 0.5396, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 303 : loss = 0.5394, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 304 : loss = 0.5391, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 305 : loss = 0.5389, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 306 : loss = 0.5386, accuracy = 0.736 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 307 : loss = 0.5384, accuracy = 0.737 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 308 : loss = 0.5382, accuracy = 0.737 | valid_accuracy = 0.695 | lr = 0.1000
Epoch 309 : loss = 0.5379, accuracy = 0.737 | valid_accuracy = 0.695 | lr = 0.1000
Epoc

Epoch 397 : loss = 0.5263, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 398 : loss = 0.5262, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 399 : loss = 0.5261, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 400 : loss = 0.5260, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 401 : loss = 0.5259, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 402 : loss = 0.5259, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 403 : loss = 0.5258, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 404 : loss = 0.5257, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 405 : loss = 0.5256, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 406 : loss = 0.5255, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 407 : loss = 0.5254, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 408 : loss = 0.5254, accuracy = 0.743 | valid_accuracy = 0.703 | lr = 0.1000
Epoc

...

Epoch 595 : loss = 0.5158, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 596 : loss = 0.5158, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 597 : loss = 0.5157, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 598 : loss = 0.5157, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 599 : loss = 0.5156, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 600 : loss = 0.5156, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 601 : loss = 0.5156, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 602 : loss = 0.5155, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 603 : loss = 0.5155, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 604 : loss = 0.5155, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 605 : loss = 0.5154, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 606 : loss = 0.5154, accuracy = 0.748 | valid_accuracy = 0.703 | lr = 0.1000
Epoc

Epoch 694 : loss = 0.5127, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 695 : loss = 0.5127, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 696 : loss = 0.5126, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 697 : loss = 0.5126, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 698 : loss = 0.5126, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 699 : loss = 0.5126, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 700 : loss = 0.5125, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 701 : loss = 0.5125, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 702 : loss = 0.5125, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 703 : loss = 0.5125, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 704 : loss = 0.5124, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 705 : loss = 0.5124, accuracy = 0.750 | valid_accuracy = 0.703 | lr = 0.1000
Epoc

Epoch 793 : loss = 0.5102, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 794 : loss = 0.5102, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 795 : loss = 0.5102, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 796 : loss = 0.5101, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 797 : loss = 0.5101, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 798 : loss = 0.5101, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 799 : loss = 0.5101, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 800 : loss = 0.5100, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 801 : loss = 0.5100, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 802 : loss = 0.5100, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 803 : loss = 0.5100, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 804 : loss = 0.5100, accuracy = 0.751 | valid_accuracy = 0.703 | lr = 0.1000
Epoc

Epoch 892 : loss = 0.5082, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 893 : loss = 0.5082, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 894 : loss = 0.5081, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 895 : loss = 0.5081, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 896 : loss = 0.5081, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 897 : loss = 0.5081, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 898 : loss = 0.5081, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 899 : loss = 0.5080, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 900 : loss = 0.5080, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 901 : loss = 0.5080, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 902 : loss = 0.5080, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 903 : loss = 0.5080, accuracy = 0.752 | valid_accuracy = 0.703 | lr = 0.1000
Epoc

Epoch 991 : loss = 0.5064, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 992 : loss = 0.5064, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 993 : loss = 0.5064, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 994 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 995 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 996 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 997 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 998 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 999 : loss = 0.5063, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
Epoch 1000 : loss = 0.5062, accuracy = 0.753 | valid_accuracy = 0.703 | lr = 0.1000
[end training]

[testing]
loss = 0.0000 , accuracy = 0.812
